In [1]:
import os
import sys
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt

import jax
from jax import numpy as jnp
from jax.config import config
from flax.training import checkpoints
import flax.linen as nn

import nest_asyncio
nest_asyncio.apply()

from scipy.interpolate import LinearNDInterpolator
from scipy.interpolate import NearestNDInterpolator
from scipy.interpolate import interp1d

from scipy.optimize import minimize
from scipy.stats import qmc
# Constants
from scipy.constants import Boltzmann, Avogadro
kb = Boltzmann # [J/K] Boltzman's constant
Na = Avogadro  # [mol-1] Avogadro's Number
R = Na * kb    # [J mol-1 K-1] Ideal gas constant

sys.path.append("../../../../")
from python_helpers.feanneos import HelmholtzModel
from python_helpers.feanneos import helper_solver_funs, helper_jitted_funs
from python_helpers.transport_properties import TransportModel_PVT_Tinv
from python_helpers import helper_get_alpha
from python_helpers import linear_activation
from python_helpers.data_figures import mie_params_of_vle_sle_sve_visc as mie_params_of

PRECISSION = 'float64'
if PRECISSION == 'float64':
    config.update("jax_enable_x64", True)
    type_np = np.float64
    type_jax = jnp.float64
else:
    config.update("jax_enable_x32", True)
    type_np = np.float32
    type_jax = jnp.float32

np.seterr(all="ignore")

{'divide': 'ignore', 'over': 'ignore', 'under': 'ignore', 'invalid': 'ignore'}

In [2]:
#######################################
# type and optimization configuration #
#######################################

of_type = "vle_sle_sve_visc"
# ranges for sigma, eps, lambda_r
l_bounds = [2.5, 100., 9.]
u_bounds = [3.5, 300., 26.]
m_base = 5
params_file = f'optimized_mie_params_{of_type}.csv'

In [3]:
###########

filename = '../../computed_files/phase_equilibria_solid.xlsx'
excel_file = pd.ExcelFile(filename)

df_info = pd.read_excel(excel_file, sheet_name='info')
df_vle = pd.read_excel(excel_file, sheet_name='vle')
df_sle = pd.read_excel(excel_file, sheet_name='sle')
df_sve = pd.read_excel(excel_file, sheet_name='sve')

# critical point information interpolation
input_crit_interp = df_info['alpha'].to_numpy()
output_crit_interp = df_info[['rhocad_model', 'Tcad_model', 'Pcad_model']].to_numpy()
crit_interp = interp1d(input_crit_interp, output_crit_interp.T, fill_value='extrapolate')

# triple point information interpolation
input_triple_interp = df_info['alpha'].to_numpy()
output_triple_interp = df_info[['rhovad_triple', 'rholad_triple', 'rhosad_triple',
                                'T_triple', 'P_triple']].to_numpy()
triple_interp = interp1d(input_triple_interp, output_triple_interp.T, fill_value='extrapolate')

# Interpolating VLE
input_vle_interp = df_vle[['alpha', 'Tr_vle_model']].to_numpy()
output_vle_interp = df_vle[['P_vle_model', 'rhov_vle_model', 'rhol_vle_model']].to_numpy()
vle_interp = LinearNDInterpolator(input_vle_interp, output_vle_interp)

# Interpolating SLE
input_sle_interp = df_sle[['alpha', 'T_sle_model']].to_numpy()
output_sle_interp = df_sle[['P_sle_model', 'rhol_sle_model', 'rhos_sle_model']].to_numpy()
sle_interp = NearestNDInterpolator(input_sle_interp, output_sle_interp)

# Interpolating SVE
input_sve_interp = df_sve[['alpha', 'T_sve_model']].to_numpy()
output_sve_interp = df_sve[['P_sve_model', 'rhov_sve_model', 'rhos_sve_model']].to_numpy()
sve_interp = NearestNDInterpolator(input_sve_interp, output_sve_interp)

alphas_sle = np.unique(df_sle['alpha'])
Tsle_max = np.zeros_like(alphas_sle)
for i, alpha in enumerate(alphas_sle):
    Tsle_max[i] = df_sle[df_sle['alpha'] == alpha]['T_sle_model'].max()
sle_maxT_interp = interp1d(alphas_sle, Tsle_max, fill_value='extrapolate')

interpd_dict = {'crit_interp': crit_interp, 'vle_interp': vle_interp, 
                'sle_interp': sle_interp, 'sve_interp': sve_interp, 
                'triple_interp': triple_interp, 'sle_maxT_interp': sle_maxT_interp}

In [4]:
#########################
# Loading FE-ANN(s) EoS #
#########################

ckpt_folder = '../../../../3_ann_models/feanns_eos'
prefix_params = 'FE-ANN-EoS-params_'
###
Tscale = 'Tinv'
seed = 17
factor = 0.01
EPOCHS = 50000
traind_model_folder = f'models_{Tscale}_factor{factor:.2f}_seed{seed}'
ckpt_folder_model = os.path.join(ckpt_folder, traind_model_folder)
ckpt_Tinv = checkpoints.restore_checkpoint(ckpt_dir=ckpt_folder_model, target=None, prefix=prefix_params, step=EPOCHS)
helmholtz_features = list(ckpt_Tinv['features'].values())
helmholtz_model = HelmholtzModel(features=helmholtz_features)
helmholtz_params = {'params': ckpt_Tinv['params']}

fun_dic_solid = helper_jitted_funs(helmholtz_model, helmholtz_params)


#########################
# Shear visocsity model #
#########################

activation_dicts = {'linear': linear_activation, 'softplus': nn.softplus}
folder_visc = '../../../../3_ann_models/visc_models'
hidden_layers = 2
neurons = 30
prefix = 'logvisc-rho-Tinv-penalty'
ckpt_dir = folder_visc
seed = 0
features = hidden_layers * [neurons]
activation = 'linear'
params_prefix = f'{prefix}-seed{seed}-params_'
state_restored = checkpoints.restore_checkpoint(ckpt_dir=ckpt_dir, target=None, prefix=params_prefix)
params_logvisc = {'params': state_restored['params']}
logvisc_model = TransportModel_PVT_Tinv(features=features, output_activation=activation_dicts[activation])

logvisc_model_jit = jax.jit(lambda alpha, rhoad, Tad: logvisc_model.apply(params_logvisc, jnp.atleast_1d(alpha), jnp.atleast_1d(rhoad), jnp.atleast_1d(Tad)))
visc_fun = lambda alpha, rhoad, Tad: jnp.exp(logvisc_model_jit(alpha, rhoad, Tad))


In [5]:
# Reading data from NIST
filename = 'xenon.xlsx'
DataFile = pd.ExcelFile(filename)

if not os.path.exists(params_file):
    sampler = qmc.Sobol(d=3, scramble=False)
    sample = sampler.random_base2(m=m_base)

    scaled_sample = qmc.scale(sample, l_bounds, u_bounds)

    sigma_guess = scaled_sample[:, 0]
    epsilon_guess = scaled_sample[:, 1]
    lr_guess = scaled_sample[:, 2]

    n = len(sigma_guess)

    index = np.arange(n)
    solved = np.zeros(n, dtype=bool)
    sigma_sol = np.nan * np.ones(n)
    epsilon_sol = np.nan * np.ones(n)
    lr_sol = np.nan * np.ones(n)
    of = np.nan * np.ones(n)
    solved_success = np.zeros(n, dtype=bool)

    df = pd.DataFrame({'index': index, 
                    'sigma_guess': sigma_guess, 'epsilon_guess': epsilon_guess, 'lr_guess': lr_guess,
                    'optimized': solved, 
                    'sigma_sol': sigma_sol, 'epsilon_sol': epsilon_sol, 'lr_sol': lr_sol, 'of': of, 'solved_success': solved_success})
    
    df.to_csv(params_file, index=False)


In [6]:
# bounds to control sigma, epsilon and lr
bounds = ((None, None), (None, None), (7., 34.))

kwargs = {'DataFile': DataFile,
          'fun_dic': fun_dic_solid,
          'visc_fun': visc_fun,
          'interpd_dict': interpd_dict,
          'lambda_a': 6.,
          'weight_rhov_vle':0.0,
          'weight_sle': 1e-2,
          'weight_sve': 1.,
          'weight_enthalpy': 1.0,
          'weight_visc': 1.0,
          'add_critical_point': True,    
          'add_triple_point': True,
          }
args = tuple(kwargs.values())

In [7]:
df = pd.read_csv(params_file)
n = len(df)
for i in range(n):
    if not df.loc[i, 'optimized']:
        inc0 = df.loc[i, ['sigma_guess', 'epsilon_guess', 'lr_guess']].to_list()
        sol = minimize(mie_params_of, inc0, method='Nelder-Mead', args=args, bounds=bounds, options={'maxiter':10})

        df.loc[i, 'solved_success'] = sol.success
        df.loc[i, 'sigma_sol'] = sol.x[0]
        df.loc[i, 'epsilon_sol'] = sol.x[1]
        df.loc[i, 'lr_sol'] = sol.x[2]
        df.loc[i, 'of'] = sol.fun
        df.loc[i, 'optimized'] = True

        df.to_csv(params_file, index=False)
    print(i, df.loc[i, 'optimized'], df.loc[i, 'of'])

0 True 7.309458903195519
1 True 0.4163793982741046
2 True 0.7787463785393585
3 True 2.28105728496945
4 True 1.4761054674923788
5 True 0.0152486299469838
6 True 6.235670791286812
7 True 2.2717079422204285
8 True 1.0782995925314889
9 True 0.0152486299444343
10 True 6.177820860488692
11 True 0.3524493992080105
12 True 6.323200157966361
13 True 0.9841471003996396
14 True 1.1489829271897032
15 True 0.4109284284350492
16 True 0.9349474291311826
17 True 1.5841836672514424
18 True 6.407502248390781
19 True 1.7605727481119593
20 True 6.231518566068637
21 True 0.0152486299446472
22 True 0.0152486299439173
23 True 1.722579427440472
24 True 6.368505080026675
25 True 0.4425319076523077
26 True 0.0152486299439798
27 True 0.5546641518350923
28 True 1.627198417203319
29 True 0.4748651462528066
30 True 6.222072488417055
31 True 3.7988194441863614


In [8]:
df = pd.read_csv(params_file)
n = len(df)

of_order = np.argsort(df['of'].to_numpy())

for i in of_order[:5]:
    if not df.loc[i, 'solved_success']:
        inc0 = df.loc[i, ['sigma_sol', 'epsilon_sol', 'lr_sol']].to_list()

        sol = minimize(mie_params_of, inc0, method='Nelder-Mead', args=args, bounds=bounds)

        df.loc[i, 'solved_success'] = sol.success
        df.loc[i, 'sigma_sol'] = sol.x[0]
        df.loc[i, 'epsilon_sol'] = sol.x[1]
        df.loc[i, 'lr_sol'] = sol.x[2]
        df.loc[i, 'of'] = sol.fun
        df.loc[i, 'optimized'] = True

        df.to_csv(params_file, index=False)


In [9]:
of_order = np.argsort(df['of'].to_numpy())
df.loc[of_order].head(n=10)

,index,sigma_guess,epsilon_guess,lr_guess,optimized,sigma_sol,epsilon_sol,lr_sol,of,solved_success
22,22,3.21875,168.75,19.09375,True,3.985446,235.880768,13.008352,0.015249,True
26,26,3.40625,181.25,22.28125,True,3.985447,235.880789,13.008356,0.015249,True
9,9,3.18750,262.50,16.43750,True,3.985447,235.880816,13.008360,0.015249,True
21,21,3.46875,218.75,14.84375,True,3.985446,235.880794,13.008355,0.015249,True
5,5,3.37500,275.00,11.12500,True,3.985448,235.880783,13.008360,0.015249,True
11,11,2.93750,212.50,12.18750,True,3.564892,222.687757,10.692901,0.352449,False
15,15,2.56250,287.50,18.56250,True,3.722479,230.177469,10.969444,0.410928,False
1,1,3.00000,200.00,17.50000,True,3.544444,240.740741,12.898148,0.416379,False
25,25,3.15625,231.25,9.53125,True,3.962847,211.979167,10.060764,0.442532,False
29,29,3.28125,256.25,20.15625,True,4.031647,257.872799,16.586280,0.474865,False


In [10]:
param_best = df.loc[of_order[0], ['sigma_sol', 'epsilon_sol', 'lr_sol']].to_list()
param_best

[3.9854460278136736, 235.8807684669436, 13.008352330662396]